In [121]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.impute import KNNImputer

In [122]:
data_test = pd.read_csv('credit_test.csv')
data_train = pd.read_csv('credit_train.csv')

data_train.head()

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,14dd8831-6af5-400b-83ec-68e61888a048,981165ec-3274-42f5-a3b4-d104041a9ca9,Fully Paid,445412,Short Term,709.0,1167493.0,8 years,Home Mortgage,Home Improvements,5214.74,17.2,NaN,6,1,228190,416746.0,1.0,0.0
1,4771cc26-131a-45db-b5aa-537ea4ba5342,2de017a3-2e01-49cb-a581-08169e83be29,Fully Paid,262328,Short Term,NaN,NaN,10+ years,Home Mortgage,Debt Consolidation,33295.98,21.1,8.0,35,0,229976,850784.0,0.0,0.0
2,4eed4e6a-aa2f-4c91-8651-ce984ee8fb26,5efb2b2b-bf11-4dfd-a572-3761a2694725,Fully Paid,99999999,Short Term,741.0,2231892.0,8 years,Own Home,Debt Consolidation,29200.53,14.9,29.0,18,1,297996,750090.0,0.0,0.0
3,77598f7b-32e7-4e3b-a6e5-06ba0d98fe8a,e777faab-98ae-45af-9a86-7ce5b33b1011,Fully Paid,347666,Long Term,721.0,806949.0,3 years,Own Home,Debt Consolidation,8741.90,12.0,NaN,9,0,256329,386958.0,0.0,0.0
4,d4062e70-befa-4995-8643-a0de73938182,81536ad9-5ccf-4eb8-befb-47a4d608658e,Fully Paid,176220,Short Term,NaN,NaN,5 years,Rent,Debt Consolidation,20639.70,6.1,NaN,15,0,253460,427174.0,0.0,0.0


In [123]:
cat_colums = []

for col in data_train.columns:
    if data_train[col].dtype == 'object':
        cat_colums.append(col)

cat_colums = cat_colums[2:]
print(cat_colums)

['Loan Status', 'Term', 'Years in current job', 'Home Ownership', 'Purpose']


In [124]:
le = LabelEncoder()

for cat_col in cat_colums:
    data_train[cat_col] = le.fit_transform(data_train[cat_col])

data_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Loan ID                       100000 non-null  object 
 1   Customer ID                   100000 non-null  object 
 2   Loan Status                   100000 non-null  int32  
 3   Current Loan Amount           100000 non-null  int64  
 4   Term                          100000 non-null  int32  
 5   Credit Score                  80846 non-null   float64
 6   Annual Income                 80846 non-null   float64
 7   Years in current job          100000 non-null  int32  
 8   Home Ownership                100000 non-null  int32  
 9   Purpose                       100000 non-null  int32  
 10  Monthly Debt                  100000 non-null  float64
 11  Years of Credit History       100000 non-null  float64
 12  Months since last delinquent  46859 non-null 

In [125]:
data_train.drop('Loan ID', axis=1, inplace=True)
data_train.drop('Customer ID', axis=1, inplace=True)

In [126]:
inputer = KNNImputer(n_neighbors=5)

data_train = pd.DataFrame(inputer.fit_transform(data_train), columns=data_train.columns)

data_train.head()

,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,1.0,445412.0,1.0,709.0,1167493.0,8.0,1.0,5.0,5214.74,17.2,23.2,6.0,1.0,228190.0,416746.0,1.0,0.0
1,1.0,262328.0,1.0,740.4,1470550.6,1.0,1.0,3.0,33295.98,21.1,8.0,35.0,0.0,229976.0,850784.0,0.0,0.0
2,1.0,99999999.0,1.0,741.0,2231892.0,8.0,2.0,3.0,29200.53,14.9,29.0,18.0,1.0,297996.0,750090.0,0.0,0.0
3,1.0,347666.0,0.0,721.0,806949.0,3.0,2.0,3.0,8741.90,12.0,53.4,9.0,0.0,256329.0,386958.0,0.0,0.0
4,1.0,176220.0,1.0,723.2,984659.8,5.0,3.0,3.0,20639.70,6.1,56.6,15.0,0.0,253460.0,427174.0,0.0,0.0


In [127]:
sc = StandardScaler()
cols = ['Current Loan Amount', 'Credit Score', 'Annual Income', 'Monthly Debt', 'Current Credit Balance', 'Maximum Open Credit']
for col in cols:  
    data_train[col] = sc.fit_transform(data_train[[col]])

data_train.head()

,Loan Status,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,1.0,-0.356000,1.0,-0.276969,-0.199105,8.0,1.0,5.0,-1.088932,17.2,23.2,6.0,1.0,-0.176642,-0.041033,1.0,0.0
1,1.0,-0.361761,1.0,-0.253925,0.101195,1.0,1.0,3.0,1.217548,21.1,8.0,35.0,0.0,-0.171894,0.010734,0.0,0.0
2,1.0,2.776244,1.0,-0.253485,0.855607,8.0,2.0,3.0,0.881165,14.9,29.0,18.0,1.0,0.008928,-0.001276,0.0,0.0
3,1.0,-0.359076,0.0,-0.268163,-0.556367,3.0,2.0,3.0,-0.799225,12.0,53.4,9.0,0.0,-0.101838,-0.044586,0.0,0.0
4,1.0,-0.364470,1.0,-0.266548,-0.380274,5.0,3.0,3.0,0.178012,6.1,56.6,15.0,0.0,-0.109465,-0.039790,0.0,0.0


In [128]:
data_test.drop('Customer ID', axis=1, inplace=True)

In [129]:
X = data_train.drop('Loan Status', axis=1)
y = data_train[['Loan Status']]

In [130]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

In [131]:
from hyperopt import hp, fmin, tpe, Trials

space = {
    'learning_rate': hp.quniform('learning_rate', 0.01, 0.1, 0.01),
    'depth': hp.quniform('depth', 2, 10, 1),
    'iterations': hp.quniform('iterations', 50, 200, 10),
    'l2_leaf_reg': hp.quniform('l2_leaf_reg', 0.1, 10, 0.1),
    'border_count': hp.quniform('border_count', 32, 255, 32),
    'thread_count': hp.choice('thread_count', [-1, 1, 2, 4, 8])
}

def optimize_catBoost(params):
    model = CatBoostClassifier(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    loss = 1 - accuracy_score(y_test, y_pred)
    return loss

trials = Trials()
best = fmin(optimize_catBoost, space, algo=tpe.suggest, max_evals=50, trials=trials)

print(best)

0:	learn: 0.6846709	total: 48.1ms	remaining: 3.8s     

1:	learn: 0.6765188	total: 73.5ms	remaining: 2.86s    

2:	learn: 0.6685144	total: 90.9ms	remaining: 2.33s    

3:	learn: 0.6611332	total: 117ms	remaining: 2.22s     

4:	learn: 0.6538605	total: 143ms	remaining: 2.14s     

5:	learn: 0.6471365	total: 172ms	remaining: 2.12s     

6:	learn: 0.6404816	total: 198ms	remaining: 2.07s     

7:	learn: 0.6338709	total: 215ms	remaining: 1.94s     

8:	learn: 0.6275917	total: 241ms	remaining: 1.9s      

9:	learn: 0.6215116	total: 267ms	remaining: 1.87s     

10:	learn: 0.6157578	total: 289ms	remaining: 1.81s    

11:	learn: 0.6101493	total: 315ms	remaining: 1.78s    

12:	learn: 0.6047494	total: 343ms	remaining: 1.76s    

13:	learn: 0.5995371	total: 370ms	remaining: 1.74s    

14:	learn: 0.5944640	total: 397ms	remaining: 1.72s    

15:	learn: 0.5896201	total: 425ms	remaining: 1.7s     

16:	learn: 0.5850077	total: 452ms	remaining: 1.68s    

17:	learn: 0.5804266	total: 477ms	remaining: 1.6

KeyboardInterrupt: 

In [132]:
best = {'border_count': 192.0, 'depth': 7.0, 'iterations': 170.0, 'l2_leaf_reg': 4.6000000000000005, 'learning_rate': 0.1, 'thread_count': 3}

In [133]:
from sklearn.model_selection import KFold

clfs = []
scores = []

kf = KFold(n_splits=10, shuffle=True, random_state=7575)

for num, (train_index, test_index) in enumerate(kf.split(X)):
    X_train_1, X_test_1 = X.iloc[train_index], X.iloc[test_index]
    y_train_1, y_test_1 = y.iloc[train_index], y.iloc[test_index]

    best_model_catboost = CatBoostClassifier(**best, verbose=150)
    clfs.append(best_model_catboost)
    best_model_catboost.fit(X_train_1, y_train_1['Loan Status'])

    y_pred_1 = best_model_catboost.predict(X_test_1)
    score = accuracy_score(y_test_1['Loan Status'], y_pred_1)
    scores.append(score)
    print(f"fold: {num} acc: {score}")

print("mean accuracy score --", np.mean(scores, dtype="float16"), np.std(scores).round(4))

0:	learn: 0.6516412	total: 17.1ms	remaining: 2.88s
150:	learn: 0.4133310	total: 2.11s	remaining: 265ms
169:	learn: 0.4125005	total: 2.34s	remaining: 0us
fold: 0 acc: 0.8193
0:	learn: 0.6516682	total: 13.2ms	remaining: 2.23s
150:	learn: 0.4135162	total: 1.93s	remaining: 243ms
169:	learn: 0.4124375	total: 2.16s	remaining: 0us
fold: 1 acc: 0.8201
0:	learn: 0.6515478	total: 11.7ms	remaining: 1.97s
150:	learn: 0.4137832	total: 1.87s	remaining: 236ms
169:	learn: 0.4126519	total: 2.1s	remaining: 0us
fold: 2 acc: 0.8143
0:	learn: 0.6516637	total: 12.6ms	remaining: 2.13s
150:	learn: 0.4135605	total: 1.91s	remaining: 240ms
169:	learn: 0.4124935	total: 2.13s	remaining: 0us
fold: 3 acc: 0.8196
0:	learn: 0.6517951	total: 13.5ms	remaining: 2.28s
150:	learn: 0.4142983	total: 1.96s	remaining: 246ms
169:	learn: 0.4134573	total: 2.18s	remaining: 0us
fold: 4 acc: 0.8219
0:	learn: 0.6516194	total: 12ms	remaining: 2.02s
150:	learn: 0.4127913	total: 1.95s	remaining: 246ms
169:	learn: 0.4119526	total: 2.18s	

In [134]:
cat_colums = []

for col in data_test.columns[1:]:
    if data_test[col].dtype == 'object':
        cat_colums.append(col)


print(cat_colums)

for cat_col in cat_colums:
    data_test[cat_col] = le.fit_transform(data_test[cat_col])


['Term', 'Years in current job', 'Home Ownership', 'Purpose']


In [135]:
data_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 17 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Loan ID                       10000 non-null  object 
 1   Current Loan Amount           10000 non-null  int64  
 2   Term                          10000 non-null  int32  
 3   Credit Score                  8019 non-null   float64
 4   Annual Income                 8019 non-null   float64
 5   Years in current job          10000 non-null  int32  
 6   Home Ownership                10000 non-null  int32  
 7   Purpose                       10000 non-null  int32  
 8   Monthly Debt                  10000 non-null  float64
 9   Years of Credit History       10000 non-null  float64
 10  Months since last delinquent  4694 non-null   float64
 11  Number of Open Accounts       10000 non-null  int64  
 12  Number of Credit Problems     10000 non-null  int64  
 13  Cu

In [136]:
data_test.iloc[:, 1:] = pd.DataFrame(inputer.fit_transform(data_test.iloc[:, 1:]), columns=data_test.columns[1:])

data_test.head()

C:\Users\Master\AppData\Local\Temp\ipykernel_10928\2872598347.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0         611314.0
1         266662.0
2         153494.0
3         176242.0
4         321992.0
           ...    
9995      157806.0
9996      132550.0
9997      223212.0
9998    99999999.0
9999    99999999.0
Name: Current Loan Amount, Length: 10000, dtype: float64' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  data_test.iloc[:, 1:] = pd.DataFrame(inputer.fit_transform(data_test.iloc[:, 1:]), columns=data_test.columns[1:])
C:\Users\Master\AppData\Local\Temp\ipykernel_10928\2872598347.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
9995    1.0
9996    1.0
9997    0.0
9998    1.0
9999    1.0
Name: Term, Length: 10000,

,Loan ID,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,f738779f-c726-40dc-92cf-689d73af533d,611314.0,1.0,747.0,2074116.0,1.0,1.0,3.0,42000.83,21.8,28.0,9.0,0.0,621908.0,1058970.0,0.0,0.0
1,6dcc0947-164d-476c-a1de-3ae7283dde0a,266662.0,1.0,734.0,1919190.0,1.0,1.0,3.0,36624.40,19.4,29.0,11.0,0.0,679573.0,904442.0,0.0,0.0
2,f7744d01-894b-49c3-8777-fc6431a2cff1,153494.0,1.0,709.0,871112.0,2.0,3.0,3.0,8391.73,12.5,10.0,10.0,0.0,38532.0,388036.0,0.0,0.0
3,83721ffb-b99a-4a0f-aea5-ef472a138b41,176242.0,1.0,727.0,780083.0,1.0,3.0,3.0,16771.87,16.5,27.0,16.0,1.0,156940.0,531322.0,1.0,0.0
4,08f3789f-5714-4b10-929d-e1527ab5e5a3,321992.0,1.0,744.0,1761148.0,1.0,1.0,3.0,39478.77,26.0,44.0,14.0,0.0,359765.0,468072.0,0.0,0.0


In [137]:
for col in cols:  
    data_test[col] = sc.fit_transform(data_test[[col]])

data_test.head()

,Loan ID,Current Loan Amount,Term,Credit Score,Annual Income,Years in current job,Home Ownership,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,f738779f-c726-40dc-92cf-689d73af533d,-0.347880,1.0,-0.254511,0.880972,1.0,1.0,3.0,1.901002,21.8,28.0,9.0,0.0,0.853223,0.199774,0.0,0.0
1,6dcc0947-164d-476c-a1de-3ae7283dde0a,-0.358787,1.0,-0.264060,0.690121,1.0,1.0,3.0,1.467395,19.4,29.0,11.0,0.0,1.001787,0.116752,0.0,0.0
2,f7744d01-894b-49c3-8777-fc6431a2cff1,-0.362368,1.0,-0.282423,-0.600991,2.0,3.0,3.0,-0.809555,12.5,10.0,10.0,0.0,-0.649745,-0.160691,0.0,0.0
3,83721ffb-b99a-4a0f-aea5-ef472a138b41,-0.361649,1.0,-0.269201,-0.713128,1.0,3.0,3.0,-0.133701,16.5,27.0,16.0,1.0,-0.344687,-0.083710,1.0,0.0
4,08f3789f-5714-4b10-929d-e1527ab5e5a3,-0.357036,1.0,-0.256714,0.495431,1.0,1.0,3.0,1.697599,26.0,44.0,14.0,0.0,0.177857,-0.117691,0.0,0.0


In [138]:
y_pred_1 = np.zeros((data_test.shape[0], len(clfs[0].classes_)))
#predict = best_model_catboost.predict(data_test.iloc[:, 1:])
for n, clf in enumerate(clfs):
    y_pred_1 += clf.predict_proba(data_test.iloc[:, 1:])

y_pred_1_class = np.argmax(y_pred_1, axis=1)


In [139]:
print(y_pred_1_class)

[1 1 1 ... 1 1 1]


In [140]:
out_col = ['Loan ID', 'Loan Status']
output = pd.DataFrame(data_test['Loan ID'], columns=out_col)
output['Loan Status'] = y_pred_1_class
output.head()

,Loan ID,Loan Status
0,f738779f-c726-40dc-92cf-689d73af533d,1
1,6dcc0947-164d-476c-a1de-3ae7283dde0a,1
2,f7744d01-894b-49c3-8777-fc6431a2cff1,1
3,83721ffb-b99a-4a0f-aea5-ef472a138b41,1
4,08f3789f-5714-4b10-929d-e1527ab5e5a3,1


In [141]:
output['Loan Status'] = output['Loan Status'].map({1.0: 'Fully Paid', 0.0: 'Charged Off'})
output.head()

,Loan ID,Loan Status
0,f738779f-c726-40dc-92cf-689d73af533d,Fully Paid
1,6dcc0947-164d-476c-a1de-3ae7283dde0a,Fully Paid
2,f7744d01-894b-49c3-8777-fc6431a2cff1,Fully Paid
3,83721ffb-b99a-4a0f-aea5-ef472a138b41,Fully Paid
4,08f3789f-5714-4b10-929d-e1527ab5e5a3,Fully Paid


In [142]:
output.to_csv('submission.csv', index=False)

In [76]:

output = pd.DataFrame({
    'Loan ID': data_test['Loan ID'],
    'Loan Status': predict
})

output['Loan Status'] = output['Loan Status'].map({1.0: 'Fully Paid', 0.0: 'Charged Off'})
output.head()
output.to_csv('submission.csv', index=False)